# Basic QIIME2 workflow

In [1]:
#QIIME2 outputs two type of files: .qza fles and .qzv files
#More information can be found at docs.qiime2.com/2019.7

In [ ]:

#make a directory for all of your fastq files to live in and move your files there
!mkdir fastq_files
!mv *.{fastq,fq,fastq.gz} fastq_files

In [ ]:
#convert your fastq files to a format the qiime2 can use

#IMPORTANT: the --type flag refers to the type of fastq files you have. 
#See https://docs.qiime2.org/2019.7/tutorials/importing/ for which type of fastq file you have

!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path fastq_files \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path fastq_files/demux.qza

In [ ]:
#The type of fastq shown here is already demutliplexed and does not require another step. 
#However, if you are using a different type of fastq you may need to perform a demultiplexing step.
#Here is an example:
#qiime demux emp-single \
#  --i-seqs emp-single-end-sequences.qza \
#  --m-barcodes-file sample-metadata.tsv \
#  --m-barcodes-column BarcodeSequence \
#  --o-per-sample-sequences demux.qza

In [ ]:
#View your stats from your demutliplexed sequences

!qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

In [ ]:
#View the stats
!qiime tools view demux.qzv

In [ ]:
#dada2
#Include bit on trimming vs trunc
#must look at demux graphs to know where to trunc
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

In [ ]:
#view denoising stats

!qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

In [ ]:
#view table

!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv


In [ ]:
##You may want to filter out mitochondria and chloroplasts out of your analysis
#Use this table for all downstream commands

!qiime taxa filter-table \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-exclude mitochondria,chloroplast \
  --o-filtered-table table-no-mitochondria-no-chloroplast.qza

In [ ]:
#view representitive sequences
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

In [ ]:
#assign taxonomy
#greengenes
#If you want to use SILVA, you can choose a different classifier here: https://docs.qiime2.org/2019.7/data-resources/

#download the classifier
!wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza"

!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza



In [ ]:
#view taxonomy

!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

In [ ]:
#generate a taxa barplot

!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

In [ ]:
#generate a tree

!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

In [ ]:
#core diversity metrics

##SAMPLING DEPTH NEEDS TO BE CHOSEN USING TABLE.QZV

!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 1109 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results